# 요구사항 1

In [25]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import pandas as pd
import os

from pathlib import Path
BASE_PATH = "/Users/leodi/git/link_dl" # BASE_PATH: /Users/yhhan/git/link_dl
print("BASE_PATH:", BASE_PATH)

import sys
sys.path.append(BASE_PATH)

# 기존 캘리포니아 데이터셋 대신 과제에 맞는 타이타닉 데이터셋 가져옴
from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset


def get_data():
  #기존에는 훈련/검증 데이터셋만 사용했지만, 최종 제출을 위해 테스트 데이터셋까지 모두 불러옴
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(train_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ReLU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


def get_model_and_optimizer():
  # 타이타닉 데이터의 feature 개수 10개와 클래스 개수(사망/생존) 2개에 맞게 모델 입출력 크기 수정
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()
  # 회귀 문제용 MSELoss 대신, 분류 문제에 적합한 CrossEntropyLoss로 변경
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target']
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval() # 평가 모드로 설정하여 정확한 성능 측정
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target']
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations,
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100


def main(config):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  wandb.init(
    mode="online" if config['wandb'] else "disabled",
    project="my_model_training_titanic",
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(wandb.config)

  train_data_loader, validation_data_loader, _ = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":

  # argparse 대신, Jupyter Notebook에서 실험하기 편하도록 config 딕셔너리 사용
  config = {
      "wandb": True,
      "batch_size": 16,
      "epochs": 1_000,
      "learning_rate": 0.01,
      "n_hidden_unit_list": [20, 20]
  }

  main(config)

BASE_PATH: /Users/leodi/git/link_dl


{'wandb': True, 'batch_size': 16, 'epochs': 1000, 'learning_rate': 0.01, 'n_hidden_unit_list': [20, 20]}


C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇██
Training loss,█▇▆▆▅▅▅▄▄▅▅▃▄▃▂▂▃▃▂▃▃▂▂▃▂▃▂▂▂▂▂▂▂▂▂▁▂▁▂▂
Validation loss,▃▃▃▃▅▂▁▁▂▁▂█▃▂▁▁▄▂▂▃▃▃▄▃▃▅▆█▄▅▄▄▅▄▅▃▄▄▄▅
Epoch,1000
Training loss,0.37673
Validation loss,0.59914


## 기술적 사항 / 고찰 내용
요구사항1은 회귀 문제용으로 설계된 c_my_model_training_with_argparse_wandb.py 코드를  Titanic 생존자 예측이라는 분류 문제에 맞게 변환하는 것이었습니다. 이를 위해 먼저 데이터 로딩 부분을 titanic_dataset.py를 사용하도록 수정했으며, 데이터의 특성 10개와 목표 2개에 맞춰 모델의 입력과 출력을 각각 10과 2로 변경했습니다. 그리고 문제 유형이 회귀에서 분류로 바뀜에 따라서 손실 함수를 기존의 nn.MSELoss에서 분류 문제에 적합한 nn.CrossEntropyLoss로 교체했습니다. 마지막으로, argparse 모듈을 제거하고 Jupyter Notebook 환경에서의 실험을 위해 config 딕셔너리 방식으로 변경했습니다. 또한, 내용을 wandb를 통해 기록하여  분석의 기반을 마련했습니다.

# 요구사항 2

In [31]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import pandas as pd
import os

from pathlib import Path
BASE_PATH = "/Users/leodi/git/link_dl" # BASE_PATH: /Users/yhhan/git/link_dl
print("BASE_PATH:", BASE_PATH)

import sys
sys.path.append(BASE_PATH)

from _03_homeworks.homework_2.titanic_dataset import get_preprocessed_dataset


def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(train_dataset, validation_dataset, test_dataset)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    # 비교를 위하여 Sigmoid, ReLU, ELU, Leaky ReLU로 변경해 가면서 실험
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
  next_print_epoch = 100

  lowest_validation_loss = float('inf')
  lowest_validation_loss_epoch = 0

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target']
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target']
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    avg_loss_validation = loss_validation / num_validations
    avg_loss_train = loss_train / num_trains

    # 가장 낮은 validation loss와 그때의 epoch을 기록
    if avg_loss_validation < lowest_validation_loss:
        lowest_validation_loss = avg_loss_validation
        lowest_validation_loss_epoch = epoch

    wandb.log({
      "Epoch": epoch,
      "Training loss": avg_loss_train,
      "Validation loss": avg_loss_validation,
      "Lowest Validation loss": lowest_validation_loss,
      "Lowest Validation loss epoch": lowest_validation_loss_epoch,
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {avg_loss_train:.4f}, "
        f"Validation loss {avg_loss_validation:.4f}"
      )
      next_print_epoch += 100


def main(config):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  wandb.init(
    mode="online" if config['wandb'] else "disabled",
    project="my_model_training_titanic",
    notes="My first wandb experiment",
    tags=["my_model", "titanic"],
    name=current_time_str,
    config=config
  )
  print(wandb.config)

  train_data_loader, validation_data_loader, _ = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":

  config = {
      "wandb": True,
      "batch_size": 128,
      "epochs": 1_000,
      "learning_rate": 0.01,
      "n_hidden_unit_list": [20, 20]
  }

  main(config)

BASE_PATH: /Users/leodi/git/link_dl


{'wandb': True, 'batch_size': 128, 'epochs': 1000, 'learning_rate': 0.01, 'n_hidden_unit_list': [20, 20]}


C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
Lowest Validation loss,█████▇▇▇▆▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Lowest Validation loss epoch,▁▁▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████████
Training loss,██▇▇▇▆▇▆▆▆▅▄▆▅▄▅▅▅▃▃▃▅▄▃▂▅▃▂▃▁▂▂▅▁▃▂▄▃▁▃
Validation loss,▄▄▃▃▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▃▁▁▃▁▃▂▂▁▂▁▃▂▁▂█▃▂▁▂▁
Epoch,1000
Lowest Validation loss,0.42472
Lowest Validation loss epoch,815
Training loss,0.44158
Validation loss,0.54197


## 기술적 사항 / 고찰 내용

![이미지](https://i.imgur.com/aUvvLA6.png)

![이미지2](https://i.imgur.com/pnlVhtl.png)

요구사항 2는 모델의 성능을 최적화하기 위해서 Activation Function과 Batch Size 두 가지를 체계적으로 실험하고 최적의 조합을 선택하는 내용이었습니다.
먼저 배치 사이즈를 16으로 고정한 상태에서 Sigmoid, ReLU, ELU, Leaky ReLU 네 가지 Activation Function에 대한 성능 비교 실험을 진행했습니다. 첨부된 Wandb 그래프에서 확인할 수 있듯이, 각 활성화 함수는 훈련 과정에서 뚜렷한 차이를 보였습니다. 실험 결과, ELU가 가장 낮은 Validation loss 값을 기록해 최고의 성능을 보였습니다.
다음으로, 가장 성능이 좋았던 ELU를 고정한 상태에서 배치 사이즈를 16, 32, 64, 128로 변경하며 실험을 진행했습니다. 그 결과, 배치 사이즈 32에서 가장 낮은 Validation loss를 기록하였기에 최적의 배치 사이즈로 보았습니다.너무 작은 배치 사이즈는 학습을 불안정하게 만들고, 너무 큰 배치 사이즈는 계산 비효율성과 Local Minima에 빠질 위험을 높인다는 내용이 있었는데 결과 또한 그것을 보여준다고 생각했습니다.
최종적으로 Activation Function은 ELU, Batch Size는 32를 사용할 때 가장 좋은 성능을 보임을 알 수 있었습니다.

# 요구사항 3

In [32]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
  next_print_epoch = 100

  lowest_validation_loss = float('inf')
  lowest_validation_loss_epoch = 0

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target']
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()
    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target']
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    avg_loss_validation = loss_validation / num_validations
    avg_loss_train = loss_train / num_trains

    if avg_loss_validation < lowest_validation_loss:
        lowest_validation_loss = avg_loss_validation
        lowest_validation_loss_epoch = epoch
        torch.save(model.state_dict(), "best_model.pth")
        # 현재 validation loss가 기록된 최저 validation loss보다 낮으면, 모델 파라미터를 파일로 저장

    wandb.log({
      "Epoch": epoch,
      "Training loss": avg_loss_train,
      "Validation loss": avg_loss_validation,
      "Lowest Validation loss": lowest_validation_loss,
      "Lowest Validation loss epoch": lowest_validation_loss_epoch,
    })

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {avg_loss_train:.4f}, "
        f"Validation loss {avg_loss_validation:.4f}"
      )
      next_print_epoch += 100

# 훈련만 진행하도록 변경
def main(model, optimizer, train_data_loader, validation_data_loader):
  training_loop(
      model=model,
      optimizer=optimizer,
      train_data_loader=train_data_loader,
      validation_data_loader=validation_data_loader
  )

def generate_submission(model, test_data_loader):
    model.load_state_dict(torch.load("best_model.pth")) # 훈련 중 저장된 최고 성능의 모델 파라미터를 불러옴
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for test_batch in test_data_loader:
            input_test = test_batch['input']
            output_test = model(input_test)

            # 모델의 출력에서 가장 높은 값의 인덱스(0 또는 1)를 최종 예측으로 선택
            predicted_test = torch.argmax(output_test, dim=-1)
            all_predictions.extend(predicted_test.numpy())

    # 캐글 제출 형식에 맞게 submission.csv 파일 생성
    test_csv_path = os.path.join(BASE_PATH, "_03_homeworks", "homework_2", "test.csv")
    test_df = pd.read_csv(test_csv_path)

    submission_df = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": all_predictions
    })

    submission_df.to_csv("submission.csv", index=False)


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":

  # 요구사항 2에서 확인했던 가장 성능이 좋았던 batch_size 32로 설정
  config = {
      "wandb": True,
      "batch_size": 32,
      "epochs": 1_000,
      "learning_rate": 0.01,
      "n_hidden_unit_list": [20, 20]
  }

  # Wandb 초기화
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
  wandb.init(
      mode="online" if config['wandb'] else "disabled",
      project="my_model_training_titanic",
      name=f"ELU_batch{config['batch_size']}_{current_time_str}",
      config=config
  )

  # wandb가 초기화된 상태에서 필요한 모든 데이터와 모델을 준비
  train_loader, validation_loader, test_loader = get_data()
  my_model, my_optimizer = get_model_and_optimizer()

  # 훈련을 실행하여 'best_model.pth' 파일을 생성
  main(my_model, my_optimizer, train_loader, validation_loader)

  # 훈련이 끝난 후, 저장된 최고 성능 모델을 사용하여 'submission.csv' 파일을 생성
  generate_submission(my_model, test_loader)
  wandb.finish()

C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
C:\Users\leodi\git\link_dl\_03_homeworks\homework_2\titanic_dataset.py:147: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

Epoch,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇█████
Lowest Validation loss,█▇▆▆▆▅▄▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Lowest Validation loss epoch,▁▁▂▂▂▅▆█████████████████████████████████
Training loss,██▇▇▇▅▆▅▅▆▅▄▄▄▄▃▃▄▄▃▃▃▂▃▃▂▂▂▂▃▂▃▂▂▁▂▁▃▁▁
Validation loss,▄▃▃▃▇▅▁▂▂▁▆▁▃█▄▂▁▂▁▁▃▃▃▂▃▃▆▂▃▄▁▂▁▂▃▄▃▂▃▂
Epoch,1000
Lowest Validation loss,0.45134
Lowest Validation loss epoch,182
Training loss,0.42141
Validation loss,0.60013


## 기술적 사항 / 고찰 내용
요구사항3은 이전 단계에서 찾은 가장 좋은 성능을 보이는 Activation	Function 및 Batch Size로 모델 구성을 해서 Kaggle에 제출할 submission.csv 파일을 생성하는 내용이였습니다.
먼저, 요구사항2의 실험 결과를 바탕으로, Activation	Function은 ELU, Batch Size는 32가 가장 낮은 Validation loss를 기록하여 이것을 바탕으로 최종 모델을 구성했습니다.
submission.csv 생성을 위해 어떤 시점의 모델을 사용해야 하는지에 대한 고찰이 필요한데 이전 요구사항2에서의 실험을 보면 950번째 Epoch에서 Validation Loss가 0.40059로 최저점을 기록했습니다. Early Stopping 개념에 따르면, Validation loss의 최저점은 모델의 일반화 성능이 가장 높은 지점이며, 이 시점을 넘어서면 과적합으로 인해 성능이 저하될 수 있습니다. 그렇기에 이전 요구사항2에서의 경험을 바탕으로 진행하면 950번째 Epoch에서 저장된 모델을 이용하는게 가장 합리적이라고 생각하며, 이번 요구사항3에서는 182번째가 최저점이였으며, 이곳에서 저장된 모델을 사용하여 최종 테스트를 수행하는 것이 가장 좋은 결과가 나올 것이라고 생각하고 진행했습니다.
위 내용을 바탕으로 추가 코딩을 진행했으며 Validation loss가 최저점을 기록할 때마다 torch.save(model.state_dict(), "best_model.pth") 코드를 통해 모델의 파라미터를 저장하도록 구현했습니다. 훈련이 모두 종료된 후에는 09.fcn_best_practice.pdf의 Tester 클래스 로직을 참고해서 generate_submission 함수를 구현했습니다. 그리고 Pandas 라이브러리를 활용하여 예측 결과를 PassengerId와 결합하고, submission.csv 파일로 생성했습니다.


# 요구사항 4

![캐글이미지](https://i.imgur.com/VvnP1xX.png)

마지막 요구사항4에 적힌 내용대로 요구사항3에서 구한 submission.csv를 제출해서 저의 점수 및 위치를 스크린 이미지 캡쳐해서 마무리했습니다.

# 숙제 후기
이번 과제는 본격적으로 딥러닝을 시도해보는 과제였다고 생각합니다. 그만큼 수업에서 들었던 내용을 직접 사용하면서 눈으로 확인하기에 좋았지만 처음하는 내용들이였기에 많은 시행착오를 겪어야 했습니다. 코드를 어떻게 수정해야할지 wandb에 연결한 데이터를 어떻게 비교해야 하는지 등 여러가지 요소가 있었지만 가장 기억에 남는 부분은 요구사항3에서 훈련과정 중 어느 Epoch	시점에 테스트를 수행하여 submission.csv 를 구성해야 하는지 고찰하는 부분이 가장 기억에 남습니다. 훈련을 매번 다시 할 경우Validation loss가 최저점인 Epoch 값이 점점 달라지기에 한번의 훈련으로 나온 Epoch 값만 믿고 진행해도 될지도 고민했었습니다. 최종적으로는 현재 나온 Validation loss가 최저점인 Epoch에서 테스트를 수행하게 진행했지만 고민을 많이했던만큼 기억에 남는 것 같습니다.
마지막으로, 첫 시도는 굉장히 힘들었지만 고민하면서 시도해가며 익숙해졌고 다음이 오면 더 체계적으로 진행할 수 있겠다는 생각이 들은 과제 였습니다.